# Step 1: Convert file data

## Convert Format

If file data (train/dev/test) have format like ViQuAD or SQuAD. You need to change the format of the data file so that the code can load and run.

In [ ]:
# Path to train/dev/test file of ViQuAD
path_train = "path-to-train-file" 
# Ex: path-to-train-file is /content/Train.json
path_dev = "path-to-dev-file"
path_test = "path-to-test-file"

In [ ]:
!python3 convert_format.py path_train "path-to-save-new-train-file.json" 2

where path_train is the data file with old format, "path-to-save-new-train-file.json" is the data file created after the conversion is complete and the last number is the version of the data (2 is for unanswerable question and answerable question, 1 is for answerable question only)

In [ ]:
!python3 convert_format.py path_dev "path-to-save-new-dev-file.json" 2

In [ ]:
!python3 convert_format.py path_test "path-to-save-new-test-file.json" 2

## Convert Question Word 

Apply rule convert question word for dataset

You can save the data file after converting the question word as another file for later use. But due to the restriction of generating too many files, I overwritten the old file.

In [ ]:
!python apply_question_word.py "path-to-save-new-train-file.json" "path-to-save-new-train-file.json"

Where, the first parameter is the name of the data file to be converted, the second parameter is the file to be saved after conversion

In [ ]:
!python apply_question_word.py "path-to-save-new-dev-file.json" "path-to-save-new-test-file.json"

In [ ]:
!python apply_question_word.py "path-to-save-new-test-file.json" "path-to-save-new-test-file.json"

# Step 2: Training MRC4MRC

MRC4MRC has two machine reading comprehension, so we need training two model including: MRC1 and MRC2

## Training MRC1

In [ ]:
!python3 run_qa.py \
  --model_name_or_path xlm-roberta-large \
  --train_file "path-to-save-new-train-file.json" \
  --validation_file "path-to-save-new-dev-file.json" \
  --do_train \
  --do_eval \
  --per_device_train_batch_size 4 \
  --learning_rate 2e-5 \
  --num_train_epochs 2 \
  --max_seq_length 384 \
  --max_answer_length 128 \
  --doc_stride 128 \
  --save_steps 1000 \
  --overwrite_output_dir \
  --version_2_with_negative True \
  --output_dir  './MRC1' #path-to-save-model-mrc1

## Training MRC2

In [ ]:
!python3 run_qa.py \
  --model_name_or_path xlm-roberta-large \
  --train_file "path-to-save-new-train-file.json" \
  --validation_file "path-to-save-new-dev-file.json" \
  --do_train \
  --do_eval \
  --per_device_train_batch_size 4 \
  --learning_rate 2e-5 \
  --num_train_epochs 2 \
  --max_seq_length 384 \
  --max_answer_length 128 \
  --doc_stride 128 \
  --save_steps 1000 \
  --overwrite_output_dir \
  --output_dir  './MRC2' #path-to-save-model-mrc2

# Step 3: Evaluate and Predictions

## Evaluate in MRC1

In [ ]:
!python run_qa.py \
  --model_name_or_path "./MRC1" \
  --test_file "path-to-save-new-test-file.json" \
  --do_predict \
  --per_device_train_batch_size 4 \
  --learning_rate 2e-5 \
  --num_train_epochs 2 \
  --max_seq_length 384 \
  --max_answer_length 128 \
  --doc_stride 128 \
  --save_steps 1000 \
  --overwrite_output_dir \
  --version_2_with_negative True \
  --output_dir "dir-to-save-result-mrc1". # You should save it in ./MRC1 to easy evaluate

## Evaluate in MRC2

In [ ]:
!python run_qa.py \
  --model_name_or_path "./MRC1" \
  --test_file "path-to-save-new-test-file.json" \
  --do_predict \
  --per_device_train_batch_size 4 \
  --learning_rate 2e-5 \
  --num_train_epochs 2 \
  --max_seq_length 384 \
  --max_answer_length 128 \
  --doc_stride 128 \
  --save_steps 1000 \
  --overwrite_output_dir \
  --output_dir "dir-to-save-result-mrc2". # You should save it in ./MRC2 to easy evaluate

## Combine result of MRC1 and MRC2

In [ ]:
!python3 combine_prediction.py "path-to-save-new-test-file.json" "dir-to-save-result-mrc1" "dir-to-save-result-mrc2"

# Step 4: Compute Metrics F1-score and EM

In [ ]:
!pip3 install datasets

In [ ]:
import json
from datasets import load_metric
metric = load_metric("squad_v2")

# Prediction
# MRC4MRC_prediction.json is generated after running file combine_prediction.py
with open("MRC4MRC_prediction.json") as f:
  final_predictions = json.load(f)
formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]

# Gold label
# path-to-save-new-test-file.json is created in step 1
with open("path-to-save-new-test-file.json") as f:
  data_test = json.load(f)["data"]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in data_test]

metric.compute(predictions=formatted_predictions, references=references)